In [27]:
### 資料輸入
import pandas as pd
import os

## 先創建要存放檔案名、數據框的空列表；預設put變數為空
files = []
pre = ['汽車組件', '汽車製造', '自行車組件', '金屬製品', '電子零組件(1)', '電子零組件(2)', '電子零組件(3)', '電子零組件(4)']
put = None

## 分別輸入要合併的所有資料名稱，並將名稱放到files這個空列表中，直到輸入Q才停止迴圈，進下一步驟
while True:
    put = input('請輸入要合併的文件名（輸入pre載入預設/輸入Q結束輸入）: ')
    if put == 'pre':
        files.extend(pre)
    elif put == 'Q':
        print(files)      
        confirm = input('確認資料已輸入完畢?(Y/N)') # 輸入Q後進行確認，如確定(Y)則進下一步，否(N)則可繼續輸入要合併的檔案
        if confirm == 'Y':
            break
    else:
        files.append(put)

請輸入要合併的文件名（輸入pre載入預設/輸入Q結束輸入）: pre
請輸入要合併的文件名（輸入pre載入預設/輸入Q結束輸入）: Q
['汽車組件', '汽車製造', '自行車組件', '金屬製品', '電子零組件(1)', '電子零組件(2)', '電子零組件(3)', '電子零組件(4)']
確認資料已輸入完畢?(Y/N)Y


In [3]:
def clean_data(df):
    df = df.set_index(df.columns.drop('申请人(第2语言)', errors='ignore').tolist())['申请人(第2语言)'].str.split('|', expand=True).stack().reset_index().rename(columns={0: '申请人(第2语言)'})
    df['申请人(第2语言)'] = df['申请人(第2语言)'].str.lstrip()
    df['申请人(第2语言)'] = df['申请人(第2语言)'].str.rstrip()
    df['申请人(第2语言)'] = df['申请人(第2语言)'].str.rstrip(';')
    df['申请人(第2语言)'] = df['申请人(第2语言)'].str.split(n=1).str[0]
    df = df.dropna(subset=['申请人(第2语言)'])
    df = df[df['申请人(第2语言)'].str.endswith(('公司', '企業'))]
    
    # 公司名稱替換
    df['申请人(第2语言)'] = df['申请人(第2语言)'].replace({
        '乾坤科技股份有限公司': '台達電子工業股份有限公司',
        '蘇州建通光電端子有限公司': '建通精密工業股份有限公司',
        '廣州貴冠科技有限公司': '僑威科技股份有限公司',
        '川益科技股份有限公司': '川湖科技股份有限公司'
    })
    
    return df

In [4]:
def calculate_patent_quant(file_name):
    # 讀取Excel檔案
    cleaned = pd.read_excel(file_name)

    # 指定要顯示的年份範圍（1990年到2023年）
    year_range = range(1990, 2023)

    # 獲取所有在資料中出現的公司名單
    all_companies = cleaned['申请人(第2语言)'].unique()

    # 在DataFrame中添加一個新的欄位，將公司和年份組合成一個新的欄位
    cleaned['公司_年份'] = cleaned['申请人(第2语言)'] + '_' + cleaned['申請日(年)'].astype(str)

    # 使用樞紐分析計算每家公司在每年份中出現的次數(即專利數量，使用count計算出現次數)
    pivot_quant = cleaned.pivot_table(index='申请人(第2语言)', columns='申請日(年)', values='公司_年份', aggfunc='count', fill_value=0)

    # 將所有在資料中出現的公司添加到索引中(因為不是所有公司都有進入，此步驟是讓沒進入的公司也出現在表格中，以便後續分析)
    pivot_quant = pivot_quant.reindex(all_companies, fill_value=0)

    # 重新索引以包含所有範圍內的年份(同上，不是所有年份都有公司進入，所以要加入所有研究過程的年分)
    pivot_quant = pivot_quant.reindex(columns=year_range, fill_value=0)

    # 將結果寫入Excel檔案
    output_file = file + '_數量.xlsx'
    pivot_quant.to_excel(output_file)
    print(file + "專利數量已統計至", output_file)

In [5]:
def calculate_cited_quant(file_name):
    # 讀取Excel檔案
    cleaned = pd.read_excel(file_name)

    # 將專利被引用數量轉換為整數
    cleaned['(F1)引用文献数'] = cleaned['(F1)引用文献数'].astype(int)

    # 指定要顯示的年份範圍（1990年到2023年）
    year_range = range(1990, 2023)

    # 獲取所有在資料中出現的公司名單
    all_companies = cleaned['申请人(第2语言)'].unique()

    # 使用樞紐分析計算每家公司在每年的專利被引用數量(使用sum將每份專利的被引用數量加總)
    pivot_cited = cleaned.pivot_table(index='申请人(第2语言)', columns='申請日(年)', values='(F1)引用文献数', aggfunc='sum', fill_value=0)

    # 將所有在資料中出現的公司添加到索引中(因為不是所有公司都有進入，此步驟是讓沒進入的公司也出現在表格中，以便後續分析)
    pivot_cited = pivot_cited.reindex(all_companies, fill_value=0)

    # 重新索引以包含所有範圍內的年份(同上，不是所有年份都有公司進入，所以要加入所有研究過程的年分)
    pivot_cited = pivot_cited.reindex(columns=year_range, fill_value=0)

    # 將結果寫入Excel檔案
    output_file = file + '_被引用.xlsx'
    pivot_cited.to_excel(output_file)
    print(file + "被引用數已統計至", output_file)

In [6]:
def extract_related_IPC(data_frame):
    # 定義要查找的重要IPC列表
    target_strings = ['H02P01', 'H02P03', 'H02P05', 'H02P06', 'H02P07', 'H02P09', 'H02P21', 'H02P25', 'H02P27', 'H02P29', 
                      'H02P31', 'H02K17', 'H02K19', 'H02K21', 'H02K23', 'H02K25', 'H02K27', 'H02K29', 'H02K41', 'H01M02', 
                      'H01M04/13', 'H01M04/14', 'H01M04/58', 'H01M04/82', 'H01M06/14', 'H01M06/16', 'H01M08', 'H01M10', 
                      'B60K06', 'B60L07', 'B60L11', 'B60L15', 'B60T08/17', 'G60W10', 'G60W20', 'H01M10/44', 'H01M10/46', 
                      'H02J07', 'B60L03', 'H02J', 'G01R19', 'G01R13', 'G01R31/02', 'G01R31/04', 'G01R31/06', 'G01R31/07', 
                      'G01R31/36']

    # 創建一個布林索引，找到包含目標字符串的行
    mask = data_frame['Orig. IPC(All)'].str.contains('|'.join(target_strings), case=False, na=False)

    # 使用布林索引過濾數據
    filtered_IPC = data_frame[mask]

    return filtered_IPC

In [7]:
def calculate_market_entry(file_name,data_frame):
    # 導入資料
    cleaned = pd.read_excel(file_name)
    IPC = pd.read_excel(data_frame)
    
    # 在DataFrame中添加一個新的欄位，將公司和年份組合成一個新的欄位
    IPC['公司_年份'] = IPC['申请人(第2语言)'] + '_' + IPC['申請日(年)'].astype(str)

    # 使用樞紐分析計算每家公司在每年份中出現的次數(即符合重要IPC專利出現的數量，使用count計算出現次數)
    pivot_IPC = IPC.pivot_table(index='申请人(第2语言)', columns='申請日(年)', values='公司_年份', aggfunc='count', fill_value=0)

    # 指定要顯示的年份範圍（1990年到2023年）
    year_range = range(1990, 2023)

    # 獲取所有在資料中出現的公司名單
    all_companies = cleaned['申请人(第2语言)'].unique()

    # 將所有在資料中出現的公司添加到索引中(因為不是所有公司都有進入，此步驟是讓沒進入的公司也出現在表格中，以便後續分析)
    pivot_IPC = pivot_IPC.reindex(all_companies, fill_value=0)

    # 重新索引以包含所有範圍內的年份(同上，不是所有年份都有公司進入，所以要加入所有研究過程的年分)
    pivot_IPC = pivot_IPC.reindex(columns=year_range, fill_value=0)
    
    # 將結果寫入Excel檔案
    output_file = file + '_進入新市場.xlsx'
    pivot_IPC.to_excel(output_file)
    print(file + "進入新市場已提取至", output_file)

    return pivot_IPC

In [8]:
def extract_common_patents(input_file):
    # 讀取 Excel 檔案
    df = input_file
    
    # 創建一個 ExcelWriter 來保存提取的数据到新的 Excel 文件
    output_file = ("提取出來的共同專利.xlsx")
    writer = pd.ExcelWriter(output_file, engine="xlsxwriter")

    # 創建一个工作表，命名為Part1
    part1_sheet = writer.book.add_worksheet("Part1")

    # 定義一个行索引
    row_index = 0

    # 寫入標題行
    part1_sheet.write_row(row_index, 0, df.columns.tolist())
    row_index += 1

    # 遍歷數據，提取包含 | 的行並寫入新工作表
    for index, row in df.iterrows():
        if "|" in row["申请人(第2语言)"]:
            part1_sheet.write_row(row_index, 0, row)
            row_index += 1

    # 保存新的 Excel 文件
    writer.close()

In [9]:
def clean_common_patents(input_file):
    # 讀取Excel文件
    df_co = pd.read_excel(input_file)

    # 步驟1: 將包含"|"的數據分開計算(分別計算各公司共同專利)
    df_co = df_co.set_index(df_co.columns.drop('申请人(第2语言)', errors='ignore').tolist())['申请人(第2语言)'].str.split('|', expand=True).stack().reset_index().rename(columns={0: '申请人(第2语言)'})

    # 步驟2: 移除字句最前的空格(有些專利資料的公司名稱前有空格，會影響計算)
    df_co['申请人(第2语言)'] = df_co['申请人(第2语言)'].str.lstrip()

    # 步驟3: 移除字句後的空格和内容(有些公司名稱後面會有地址或人名)
    df_co['申请人(第2语言)'] = df_co['申请人(第2语言)'].str.rstrip()

    # 步骤4: 移除字句後的分號(有些公司名稱後會有分號，影響數量計算)
    df_co['申请人(第2语言)'] = df_co['申请人(第2语言)'].str.rstrip(';')

    # 步骤5: 删除第一个空格及其後的内容(有的公司在分號拿掉後會出現空格，所以要再去一次)
    df_co['申请人(第2语言)'] = df_co['申请人(第2语言)'].str.split(n=1).str[0]

    # 步骤6: 删除空缺的公司列(在步驟一切分時是用"|"為基準，有的會切出空格)
    df_co = df_co.dropna(subset=['申请人(第2语言)'])

    # 步骤7: 只保留以「公司」、「企業」為结尾的公司名(有些為人名的資料，不在這次研究的計算範圍)
    df_co = df_co[df_co['申请人(第2语言)'].str.endswith(('公司', '企業'))]

    # 步骤8: 刪除分公司名稱(分公司共同專利若計入會引響總公司，因分公司的專利跟組公司一起申請)
    keyword = ['乾坤科技股份有限公司', '蘇州建通光電端子有限公司', '廣州貴冠科技有限公司', '川益科技股份有限公司']

    for key in keyword:
        df_co = df_co[~df_co.apply(lambda row: key in row.to_string(), axis=1)]

    # 保存清理後的數據到新的Excel文件
    df_co.to_excel('co_cleaned_data.xlsx', index=False)

In [10]:
def calculate_co_patents(file_name,input_file):
    # 讀取Excel檔案
    co_cleaned = pd.read_excel(input_file)
    cleaned = pd.read_excel(file_name)
    
    # 在DataFrame中添加一個新的欄位，將公司和年份組合成一個新的欄位
    co_cleaned['公司_年份'] = co_cleaned['申请人(第2语言)'] + '_' + co_cleaned['申請日(年)'].astype(str)

    # 使用pivot_table計算每家公司在每年份中出現的次數(即各自共同專利數量，使用count計算出現次數)
    pivot_co = co_cleaned.pivot_table(index='申请人(第2语言)', columns='申請日(年)', values='公司_年份', aggfunc='count', fill_value=0)

    # 指定要顯示的年份範圍（1990年到2023年）
    year_range = range(1990, 2023)

    # 獲取所有在資料中出現的公司名單
    all_companies = cleaned['申请人(第2语言)'].unique()

    # 將所有在資料中出現的公司添加到索引中(因為不是所有公司都有進入，此步驟是讓沒進入的公司也出現在表格中，以便後續分析)
    pivot_co = pivot_co.reindex(all_companies, fill_value=0)

    # 重新索引以包含所有範圍內的年份(同上，不是所有年份都有公司進入，所以要加入所有研究過程的年分)
    pivot_co = pivot_co.reindex(columns=year_range, fill_value=0)

    # 將結果寫入Excel檔案
    output_file = file + '_共同專利數量.xlsx'
    pivot_co.to_excel(output_file)
    print(file + "共同專利數量已統計至", output_file)

In [11]:
def delete_files(file_list):
    for file_name in file_list:
        os.remove(file_name)

In [12]:
def merge_quant_data(files):
    quant_dfs = []

    for i, file in enumerate(files, 1):
        data = str(file+'_數量.xlsx')
        var_name = f'quant_df{i}'
        quant_df = pd.read_excel(data)
        exec(f"{var_name} = quant_df")
        quant_dfs.append(quant_df)

    quant_merged = pd.concat(quant_dfs, ignore_index=True)
    quant_final = quant_merged.groupby('申请人(第2语言)').sum().reset_index()
    quant_final.to_excel('專利數量(all).xlsx', index=False)
    print('專利資料數量已合併')

In [13]:
def merge_cited_data(files):
    cited_dfs = []
    for i, file in enumerate(files, 1):
        data = str(file+'_被引用.xlsx')
        var_name = f'cited_df{i}'
        cited_df = pd.read_excel(data)
        exec(f"{var_name} = cited_df")
        cited_dfs.append(cited_df)

    cited_merged = pd.concat(cited_dfs, ignore_index=True)
    cited_final = cited_merged.groupby('申请人(第2语言)').sum().reset_index()
    cited_final.to_excel('專利被引用數(all).xlsx', index=False)
    print('專利被引用數已合併')

In [14]:
def merge_IPC_data(files):
    IPC_dfs = []
    for i, file in enumerate(files, 1):
        data = str(file+'_進入新市場.xlsx')
        var_name = f'IPC_df{i}'
        IPC_df = pd.read_excel(data)
        exec(f"{var_name} = IPC_df")
        IPC_dfs.append(IPC_df)

    IPC_merged = pd.concat(IPC_dfs, ignore_index=True)
    IPC_final = IPC_merged.groupby('申请人(第2语言)').sum().reset_index()
    IPC_final.to_excel('進入新市場(all).xlsx', index=False)
    print('進入新市場資料已合併')

In [15]:
def merge_co_data(files):
    co_dfs = []
    for i, file in enumerate(files, 1):
        data = str(file+'_共同專利數量.xlsx')
        var_name = f'co_df{i}'
        co_df = pd.read_excel(data)
        exec(f"{var_name} = co_df")
        co_dfs.append(co_df)

    co_merged = pd.concat(co_dfs, ignore_index=True)
    co_final = co_merged.groupby('申请人(第2语言)').sum().reset_index()
    co_final.to_excel('共同專利數量(all).xlsx', index=False)
    print('共同專利數量已合併')

In [16]:
def delete_all(files):
    del_all = []
    for file in files:
        del_quant = str(file+'_數量.xlsx')
        del_cited = str(file+'_被引用.xlsx')
        del_IPC = str(file+'_進入新市場.xlsx')
        del_co = str(file+'_共同專利數量.xlsx')
        del_all.extend([del_quant, del_cited, del_IPC, del_co])

    for delete in del_all:
        os.remove(delete)
    print('原資料已刪除')

In [26]:
def no_co_calculate_patent_quant(file_name):
    # 讀取Excel檔案
    cleaned = pd.read_excel(file_name)

    # 指定要顯示的年份範圍（1990年到2023年）
    year_range = range(1990, 2023)

    # 獲取所有在資料中出現的公司名單
    all_companies = cleaned['申请人(第2语言)'].unique()

    # 在DataFrame中添加一個新的欄位，將公司和年份組合成一個新的欄位
    cleaned['公司_年份'] = cleaned['申请人(第2语言)'] + '_' + cleaned['申請日(年)'].astype(str)

    # 使用樞紐分析計算每家公司在每年份中出現的次數(即專利數量，使用count計算出現次數)
    pivot_quant = cleaned.pivot_table(index='申请人(第2语言)', columns='申請日(年)', values='公司_年份', aggfunc='count', fill_value=0)

    # 將所有在資料中出現的公司添加到索引中(因為不是所有公司都有進入，此步驟是讓沒進入的公司也出現在表格中，以便後續分析)
    pivot_quant = pivot_quant.reindex(all_companies, fill_value=0)

    # 重新索引以包含所有範圍內的年份(同上，不是所有年份都有公司進入，所以要加入所有研究過程的年分)
    pivot_quant = pivot_quant.reindex(columns=year_range, fill_value=0)

    # 將結果寫入Excel檔案
    output_file = no_co_file + '_數量.xlsx'
    pivot_quant.to_excel(output_file)
    print(no_co_file + "專利數量已統計至", output_file)
    
def no_co_calculate_cited_quant(file_name):
    # 讀取Excel檔案
    cleaned = pd.read_excel(file_name)

    # 將專利被引用數量轉換為整數
    cleaned['(F1)引用文献数'] = cleaned['(F1)引用文献数'].astype(int)

    # 指定要顯示的年份範圍（1990年到2023年）
    year_range = range(1990, 2023)

    # 獲取所有在資料中出現的公司名單
    all_companies = cleaned['申请人(第2语言)'].unique()

    # 使用樞紐分析計算每家公司在每年的專利被引用數量(使用sum將每份專利的被引用數量加總)
    pivot_cited = cleaned.pivot_table(index='申请人(第2语言)', columns='申請日(年)', values='(F1)引用文献数', aggfunc='sum', fill_value=0)

    # 將所有在資料中出現的公司添加到索引中(因為不是所有公司都有進入，此步驟是讓沒進入的公司也出現在表格中，以便後續分析)
    pivot_cited = pivot_cited.reindex(all_companies, fill_value=0)

    # 重新索引以包含所有範圍內的年份(同上，不是所有年份都有公司進入，所以要加入所有研究過程的年分)
    pivot_cited = pivot_cited.reindex(columns=year_range, fill_value=0)

    # 將結果寫入Excel檔案
    output_file = no_co_file + '_被引用.xlsx'
    pivot_cited.to_excel(output_file)
    print(no_co_file + "被引用數已統計至", output_file)

def no_co_calculate_market_entry(file_name,data_frame):
    # 導入資料
    cleaned = pd.read_excel(file_name)
    IPC = pd.read_excel(data_frame)
    
    # 在DataFrame中添加一個新的欄位，將公司和年份組合成一個新的欄位
    IPC['公司_年份'] = IPC['申请人(第2语言)'] + '_' + IPC['申請日(年)'].astype(str)

    # 使用樞紐分析計算每家公司在每年份中出現的次數(即符合重要IPC專利出現的數量，使用count計算出現次數)
    pivot_IPC = IPC.pivot_table(index='申请人(第2语言)', columns='申請日(年)', values='公司_年份', aggfunc='count', fill_value=0)

    # 指定要顯示的年份範圍（1990年到2023年）
    year_range = range(1990, 2023)

    # 獲取所有在資料中出現的公司名單
    all_companies = cleaned['申请人(第2语言)'].unique()

    # 將所有在資料中出現的公司添加到索引中(因為不是所有公司都有進入，此步驟是讓沒進入的公司也出現在表格中，以便後續分析)
    pivot_IPC = pivot_IPC.reindex(all_companies, fill_value=0)

    # 重新索引以包含所有範圍內的年份(同上，不是所有年份都有公司進入，所以要加入所有研究過程的年分)
    pivot_IPC = pivot_IPC.reindex(columns=year_range, fill_value=0)
    
    # 將結果寫入Excel檔案
    output_file = no_co_file + '_進入新市場.xlsx'
    pivot_IPC.to_excel(output_file)
    print(no_co_file + "進入新市場已提取至", output_file)

    return pivot_IPC
    
def no_co_copatent(file_name):
    # 讀取Excel檔案
    cleaned = pd.read_excel(file_name)

    # 指定要顯示的年份範圍（1990年到2023年）
    year_range = range(1990, 2023)

    # 獲取所有在資料中出現的公司名單
    all_companies = cleaned['申请人(第2语言)'].unique()

    # 在DataFrame中添加一個新的欄位，將公司和年份組合成一個新的欄位
    cleaned['公司_年份'] = cleaned['申请人(第2语言)'] + '_' + cleaned['申請日(年)'].astype(str)

    # 使用樞紐分析計算每家公司在每年份中出現的次數(即專利數量，使用count計算出現次數)
    pivot_quant = cleaned.pivot_table(index='申请人(第2语言)', columns='申請日(年)', values='公司_年份', aggfunc='count', fill_value=0)

    # 將所有非0的值替換為0
    pivot_quant[pivot_quant != 0] = 0

    # 將所有在資料中出現的公司添加到索引中(因為不是所有公司都有進入，此步驟是讓沒進入的公司也出現在表格中，以便後續分析)
    pivot_quant = pivot_quant.reindex(all_companies, fill_value=0)

    # 重新索引以包含所有範圍內的年份(同上，不是所有年份都有公司進入，所以要加入所有研究過程的年分)
    pivot_quant = pivot_quant.reindex(columns=year_range, fill_value=0)

    # 將結果寫入Excel檔案
    output_file = no_co_file + '_共同專利數量.xlsx'
    pivot_quant.to_excel(output_file)
    print(no_co_file + "_共同專利數量已統計至", output_file)

In [30]:
for file in files:
    data = str(file+'.xlsx')
    df = pd.read_excel(data)
    # 清理資料
    cleaned_df = clean_data(df)
    cleaned_df.to_excel('cleaned_data.xlsx', index=False)
    # 計算專利數量
    calculate_patent_quant('cleaned_data.xlsx')
    # 計算被引用數量
    calculate_cited_quant('cleaned_data.xlsx')
    # 計算進入新市場
    extracted_df = extract_related_IPC(df)
    extracted_df.to_excel('提取相關IPC字串.xlsx', index=False)
    calculate_market_entry('cleaned_data.xlsx','提取相關IPC字串.xlsx')
    # 計算共同專利
    extract_common_patents(df)
    clean_common_patents('提取出來的共同專利.xlsx')
    calculate_co_patents('cleaned_data.xlsx','co_cleaned_data.xlsx')
    # 刪除過渡檔案
    delete_files(['cleaned_data.xlsx', '提取相關IPC字串.xlsx', '提取出來的共同專利.xlsx', 'co_cleaned_data.xlsx'])
    
print('資料清理已完成')

汽車組件專利數量已統計至 汽車組件_數量.xlsx
汽車組件被引用數已統計至 汽車組件_被引用.xlsx
汽車組件進入新市場已提取至 汽車組件_進入新市場.xlsx
汽車組件共同專利數量已統計至 汽車組件_共同專利數量.xlsx
汽車製造專利數量已統計至 汽車製造_數量.xlsx
汽車製造被引用數已統計至 汽車製造_被引用.xlsx
汽車製造進入新市場已提取至 汽車製造_進入新市場.xlsx
汽車製造共同專利數量已統計至 汽車製造_共同專利數量.xlsx
自行車組件專利數量已統計至 自行車組件_數量.xlsx
自行車組件被引用數已統計至 自行車組件_被引用.xlsx
自行車組件進入新市場已提取至 自行車組件_進入新市場.xlsx
自行車組件共同專利數量已統計至 自行車組件_共同專利數量.xlsx
金屬製品專利數量已統計至 金屬製品_數量.xlsx
金屬製品被引用數已統計至 金屬製品_被引用.xlsx
金屬製品進入新市場已提取至 金屬製品_進入新市場.xlsx
金屬製品共同專利數量已統計至 金屬製品_共同專利數量.xlsx
電子零組件(1)專利數量已統計至 電子零組件(1)_數量.xlsx
電子零組件(1)被引用數已統計至 電子零組件(1)_被引用.xlsx
電子零組件(1)進入新市場已提取至 電子零組件(1)_進入新市場.xlsx
電子零組件(1)共同專利數量已統計至 電子零組件(1)_共同專利數量.xlsx
電子零組件(2)專利數量已統計至 電子零組件(2)_數量.xlsx
電子零組件(2)被引用數已統計至 電子零組件(2)_被引用.xlsx
電子零組件(2)進入新市場已提取至 電子零組件(2)_進入新市場.xlsx
電子零組件(2)共同專利數量已統計至 電子零組件(2)_共同專利數量.xlsx
電子零組件(3)專利數量已統計至 電子零組件(3)_數量.xlsx
電子零組件(3)被引用數已統計至 電子零組件(3)_被引用.xlsx
電子零組件(3)進入新市場已提取至 電子零組件(3)_進入新市場.xlsx
電子零組件(3)共同專利數量已統計至 電子零組件(3)_共同專利數量.xlsx
電子零組件(4)專利數量已統計至 電子零組件(4)_數量.xlsx
電子零組件(4)被引用數已統計至 電子零組件(4)_被引用.xlsx
電子零組件(4)進入新市場已提取至 電子零組

In [32]:
no_co_pre = ['機車船製造']
no_co_files = []
no_co = None

no_co = input('是否要加入無共同專利資料(Y/N)') # 確定(Y)則進下一步
if no_co == 'Y':
    while True:
        put = input('請輸入無共同資料的檔案名（輸入pre載入預設/輸入Q結束輸入）: ')
        if put == 'pre':
            no_co_files.extend(no_co_pre)
        elif put == 'Q':
            print(no_co_files)      
            confirm = input('確認資料已輸入完畢?(Y/N)') # 輸入Q後進行確認，如確定(Y)則進下一步，否(N)則可繼續輸入要合併的檔案
            if confirm == 'Y':
                break
        else:
            no_co_files.append(no_co)

是否要加入無共同專利資料(Y/N)Y
請輸入無共同資料的檔案名（輸入pre載入預設/輸入Q結束輸入）: Q
[]
確認資料已輸入完畢?(Y/N)N
請輸入無共同資料的檔案名（輸入pre載入預設/輸入Q結束輸入）: pre
請輸入無共同資料的檔案名（輸入pre載入預設/輸入Q結束輸入）: Q
['機車船製造']
確認資料已輸入完畢?(Y/N)Y


In [37]:
for no_co_file in no_co_files:
    data = str(no_co_file+'.xlsx')
    no_co_df = pd.read_excel(data)
    # 清理資料
    cleaned_df = clean_data(no_co_df)
    cleaned_df.to_excel('cleaned_data.xlsx', index=False)
    # 計算專利數量
    no_co_calculate_patent_quant('cleaned_data.xlsx')
    # 計算被引用數量
    no_co_calculate_cited_quant('cleaned_data.xlsx')
    # 計算進入新市場
    extracted_df = extract_related_IPC(df)
    extracted_df.to_excel('提取相關IPC字串.xlsx', index=False)
    no_co_calculate_market_entry('cleaned_data.xlsx','提取相關IPC字串.xlsx')
    # 共同專利
    no_co_copatent('cleaned_data.xlsx')
    # 刪除過渡檔案
    delete_files(['cleaned_data.xlsx', '提取相關IPC字串.xlsx'])

print('無共同專利資料清理已完成')

機車船製造專利數量已統計至 機車船製造_數量.xlsx
機車船製造被引用數已統計至 機車船製造_被引用.xlsx
機車船製造進入新市場已提取至 機車船製造_進入新市場.xlsx
機車船製造_共同專利數量已統計至 機車船製造_共同專利數量.xlsx
無共同專利資料清理已完成


In [33]:
files = files + no_co_files

In [35]:
print("現有資料:", files)
combine = input('是否要合併資料(Y/N)') # 確定(Y)則進下一步，否則直接停止程式
if combine == 'Y':
    simple = input('合併後是否刪除原資料(Y/N)')
    if simple == 'Y':
        # 合併專利數量
        merge_quant_data(files)
        # 合併被引用數量
        merge_cited_data(files)
        # 合併進入新市場數量
        merge_IPC_data(files)
        # 合併共同專利數量
        merge_co_data(files)
        # 刪除原資料
        delete_all(files)
    else:
        merge_quant_data(files)
        merge_cited_data(files)
        merge_IPC_data(files)
        merge_co_data(files)

現有資料: ['汽車組件', '汽車製造', '自行車組件', '金屬製品', '電子零組件(1)', '電子零組件(2)', '電子零組件(3)', '電子零組件(4)', '機車船製造']
是否要合併資料(Y/N)Y
合併後是否刪除原資料(Y/N)Y
專利資料數量已合併
專利被引用數已合併
進入新市場資料已合併
共同專利數量已合併
